In [105]:
import importlib
importlib.reload(squander)


<module 'squander' from '/home/rhea_huang/research/decomposition_evaluation/squander.py'>

# generate unitary matrices for testing

In [38]:
import numpy as np

# this function generates a list of unitary matrices of dimension 2^n
# num specifies the number of matrices to be tested 
def generate_unitary_matrices(n, num, seed=None):
    if seed is not None:
        np.random.seed(seed)

    dim = 2 ** n
    unitary_matrices = []
    for _ in range(num):
        # Generate a random complex matrix of dimension 2^n x 2^n
        random_matrix = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
        # Perform QR decomposition to get a unitary matrix
        Q, _ = np.linalg.qr(random_matrix)
        unitary_matrices.append(Q)

    return unitary_matrices

In [52]:
seed = 42

array1 = generate_unitary_matrices(1, 10, seed) # a list of ten 2^1 x 2^1 unitary matrices to be evaluated
array2 = generate_unitary_matrices(2, 10, seed)
array3 = generate_unitary_matrices(3, 10, seed)
array4 = generate_unitary_matrices(4, 10, seed)
array5 = generate_unitary_matrices(5, 10, seed)

In [41]:
array4

[array([[-0.23648826+0.48221431j,  0.06829648+0.04447898j,
         -0.51150074+0.09529866j, -0.55205719+0.35974699j],
        [ 0.11148167-0.69780321j, -0.24943828+0.08410701j,
         -0.46705929+0.43192981j, -0.16220654+0.01817602j],
        [ 0.22351926+0.25918352j,  0.24364996-0.05375623j,
          0.09259829+0.53810588j, -0.27267708-0.66941977j],
        [-0.11519953+0.28596729j, -0.90164923-0.22099186j,
          0.09627758+0.13105595j,  0.0200152 -0.12766128j]]),
 array([[-0.00652005+0.16599032j, -0.38044393-0.5954188j ,
         -0.49652764-0.41565941j, -0.14846753-0.17829336j],
        [ 0.1008949 -0.32699801j, -0.60782417+0.22744342j,
          0.42585985-0.44949037j,  0.25305368-0.11948816j],
        [ 0.35672811-0.40539746j,  0.11371627-0.0396587j ,
          0.15415622-0.10998916j, -0.81118943+0.00192614j],
        [-0.71422372-0.23147279j, -0.19070434-0.16936877j,
          0.20046089+0.34819971j, -0.22692204-0.39790662j]]),
 array([[-0.53866604+0.14563203j,  0.4178465

In [49]:
# a list of 5 matrices

import pickle

with open('array1.pkl', 'rb') as f:
    array1 = pickle.load(f)

with open('array2.pkl', 'rb') as f:
    array2 = pickle.load(f)

with open('array3.pkl', 'rb') as f:
    array3 = pickle.load(f)

with open('array4.pkl', 'rb') as f:
    array4 = pickle.load(f)

# Evaluation of different methods
- Qiskit (For Qiskit decomposition, 2x2 unitary is just a U3-gate, so solovayKitaev in its library is used instead because it decomposes into basic gates like SGate, HGate, SdgGate. For larger unitaries, its module qiskit.transpile is used instead. The underlying algorithms this module used is mentioned in this post: https://quantumcomputing.stackexchange.com/questions/18157/how-efficient-is-qiskits-unitary-decomposition)
- OpenQL
- SQUANDER

## 1. 1 qubit (2x2) unitary evaluation

- qiskit Solovay Kitaev for 1q matrices

In [43]:
import solovayKitaev

result1 = []
for arr in array1:
    res = solovayKitaev.sk_decomp(arr, 1)
    result1.append(res)
 
print(result1)

averages_1q_sk = {key: sum(item[key] for item in result1) / len(result1) for key in result1[0]}

averages_1q_sk


Original circuit:
   ┌───────────────┐
q: ┤ original gate ├
   └───────────────┘

Discretized circuit:
global phase: 2.6269
   ┌───┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐»
q: ┤ Z ├┤ H ├┤ S ├┤ H ├┤ Tdg ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ Tdg ├┤ H ├┤ T ├┤ H ├»
   └───┘└───┘└───┘└───┘└─────┘└───┘└───┘└───┘└───┘└───┘└─────┘└───┘└───┘└───┘»
«   ┌─────┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌─────┐┌───┐»
«q: ┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ T ├┤ Z ├┤ H ├┤ T ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├»
«   └─────┘└───┘└─────┘└───┘└───┘└───┘└───┘└───┘└───┘└─────┘└───┘└─────┘└───┘»
«   ┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─────┐┌─────┐┌─────┐┌───┐»
«q: ┤ T ├┤ T ├┤ Z ├┤ Tdg ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ Tdg ├┤ Tdg ├┤ Tdg ├┤ H ├»
«   └───┘└───┘└───┘└─────┘└───┘└───┘└───┘└───┘└───┘└─────┘└─────┘└─────┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌─────┐
«q: ┤ T ├┤ H ├┤ T ├┤ H ├┤ Tdg ├
«   └───┘└───┘└───┘└───┘└─────┘
{'time': 0.0054552555084228516, 'total_gate': 45, 'cnot_gate': 0, 'e

{'time': 0.004606461524963379,
 'total_gate': 30.2,
 'cnot_gate': 0.0,
 'error': 1.5087160360698526}

- qiskit

In [44]:
import qiskit_decomposition

result2 = []
for arr in array1:
    res = qiskit_decomposition.qiskit_decomp(arr, 1)
    result2.append(res)
 
print(result2)

averages_1q_qiskit = {key: sum(item[key] for item in result2) / len(result2) for key in result2[0]}

averages_1q_qiskit

[{'time': 0.003081798553466797, 'total_gate': 3, 'cnot_gate': 0, 'error': 4.4147945634195513e-16}, {'time': 0.0032639503479003906, 'total_gate': 3, 'cnot_gate': 0, 'error': 5.177735492072797e-16}, {'time': 0.0024595260620117188, 'total_gate': 3, 'cnot_gate': 0, 'error': 2.6622212897024595e-16}, {'time': 0.0028412342071533203, 'total_gate': 3, 'cnot_gate': 0, 'error': 3.1309716410295113e-16}, {'time': 0.0030508041381835938, 'total_gate': 3, 'cnot_gate': 0, 'error': 8.121841189131814e-16}, {'time': 0.002928018569946289, 'total_gate': 3, 'cnot_gate': 0, 'error': 2.5133742693021536e-16}, {'time': 0.003022432327270508, 'total_gate': 3, 'cnot_gate': 0, 'error': 7.264329320467944e-16}, {'time': 0.002668619155883789, 'total_gate': 3, 'cnot_gate': 0, 'error': 6.004449063730082e-16}, {'time': 0.003526926040649414, 'total_gate': 3, 'cnot_gate': 0, 'error': 1.1857187100668868e-16}, {'time': 0.0030875205993652344, 'total_gate': 3, 'cnot_gate': 0, 'error': 4.358024505511923e-16}]


{'time': 0.0029930830001831054,
 'total_gate': 3.0,
 'cnot_gate': 0.0,
 'error': 4.483346004443512e-16}

- OpenQL

In [64]:
import openQLs

result = []
for arr in array1:
    res = openQLs.openQL_decomp(arr, 1)
    result.append(res)
 
print(result)

averages_1q_openQL = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_1q_openQL

Matrix to be factorized:
 [[-0.27702459+0.13059069j  0.76350488-0.56856287j]
 [-0.36122517-0.88074958j -0.1700831 -0.25469254j]]

Original circuit:
   ┌───────────────┐
q: ┤ original gate ├
   └───────────────┘



Discretized circuit:
   ┌─────────────┐┌─────────────┐┌────────────┐
q: ┤ Rz(-3.3412) ├┤ Ry(-2.5191) ├┤ Rz(2.3587) ├
   └─────────────┘└─────────────┘└────────────┘
Matrix to be factorized:
 [[-0.25206158+0.12990995j  0.29595727-0.91213906j]
 [-0.24880973-0.92611127j -0.24507068-0.14266042j]]

Original circuit:
   ┌───────────────┐
q: ┤ original gate ├
   └───────────────┘



Discretized circuit:
   ┌────────────┐┌─────────────┐┌───────────┐
q: ┤ Rz(2.3604) ├┤ Ry(-2.5666) ├┤ Rz(-3.44) ├
   └────────────┘└─────────────┘└───────────┘
Matrix to be factorized:
 [[-0.50622443+0.73254781j  0.45016876-0.06677282j]
 [-0.45384069+0.03375136j -0.65478445-0.60344577j]]

Original circuit:
   ┌───────────────┐
q: ┤ original gate ├
   └───────────────┘



Discretized circuit:
   ┌─────────

{'time': 0.03630585670471191,
 'total_gate': 3.0,
 'cnot_gate': 0.0,
 'error': 1.7804300880020005}

- SQUANDER
squander does not support 1 qubit gate decomposition

## 2. 2 qubits unitary evaluation

- qiskit

In [45]:
import qiskit_decomposition

result = []
for arr in array2:
    res = qiskit_decomposition.qiskit_decomp(arr, 2)
    result.append(res)
 
print(result)

averages_2q_qiskit = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_2q_qiskit

[{'time': 0.012447595596313477, 'total_gate': 23, 'cnot_gate': 3, 'error': 1.3930896426115296}, {'time': 0.012482166290283203, 'total_gate': 23, 'cnot_gate': 3, 'error': 3.7907987256971696}, {'time': 0.012616634368896484, 'total_gate': 23, 'cnot_gate': 3, 'error': 1.3272431982881805}, {'time': 0.012817144393920898, 'total_gate': 23, 'cnot_gate': 3, 'error': 3.6672454108041075}, {'time': 0.011286735534667969, 'total_gate': 23, 'cnot_gate': 3, 'error': 3.8800174081386802}, {'time': 0.011312007904052734, 'total_gate': 23, 'cnot_gate': 3, 'error': 1.5100602311931302}, {'time': 0.009934425354003906, 'total_gate': 23, 'cnot_gate': 3, 'error': 1.6276137209549089}, {'time': 0.01166224479675293, 'total_gate': 23, 'cnot_gate': 3, 'error': 3.5801638773824975}, {'time': 0.01288294792175293, 'total_gate': 23, 'cnot_gate': 3, 'error': 1.870826637383543}, {'time': 0.013332843780517578, 'total_gate': 23, 'cnot_gate': 3, 'error': 3.518131584209983}]


{'time': 0.01207747459411621,
 'total_gate': 23.0,
 'cnot_gate': 3.0,
 'error': 2.616519043666373}

- OpenQL

In [65]:
import openQLs

result = []
for arr in array2:
    res = openQLs.openQL_decomp(arr, 2)
    result.append(res)
 
print(result)

averages_2q_openQL = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_2q_openQL

Matrix to be factorized:
 [[-0.23648826+0.48221431j  0.06829648+0.04447898j -0.51150074+0.09529866j
  -0.55205719+0.35974699j]
 [ 0.11148167-0.69780321j -0.24943828+0.08410701j -0.46705929+0.43192981j
  -0.16220654+0.01817602j]
 [ 0.22351926+0.25918352j  0.24364996-0.05375623j  0.09259829+0.53810588j
  -0.27267708-0.66941977j]
 [-0.11519953+0.28596729j -0.90164923-0.22099186j  0.09627758+0.13105595j
   0.0200152 -0.12766128j]]

Original circuit:
     ┌────────────────┐
q_0: ┤0               ├
     │  original gate │
q_1: ┤1               ├
     └────────────────┘



Discretized circuit:
     ┌─────────────┐┌─────────────┐┌────────────┐                         »
q_0: ┤ Rz(0.22021) ├┤ Ry(-2.0668) ├┤ Rz(5.1878) ├──■───────────────────■──»
     ├─────────────┤└─────────────┘└────────────┘┌─┴─┐┌─────────────┐┌─┴─┐»
q_1: ┤ Rz(-1.4408) ├─────────────────────────────┤ X ├┤ Rz(-1.5143) ├┤ X ├»
     └─────────────┘                             └───┘└─────────────┘└───┘»
«     ┌────────────┐┌─────

{'time': 0.04181535243988037,
 'total_gate': 24.0,
 'cnot_gate': 6.0,
 'error': 2.8290181876935563}

- SQUANDER

In [107]:
import squander

result = []
for arr in array2:
    res = squander.squander_decomp(arr, 2)
    result.append(res)
 
print(result)

averages_2q_squander = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_2q_squander

***************************************************************
Starting to disentangle 2-qubit matrix via custom gate structure
***************************************************************


***************************************************************
Final fine tuning of the parameters in the 2-qubit decomposition
***************************************************************
The minimum with 2 layers after 0 iterations is 0.426786 calculated in 0.00269942 seconds
Reached maximal number of iterations

In the decomposition with error = 1.19158 were used 2 gates with:
4 RY opeartions,
4 RZ opeartions,
2 CNOT opeartions,

--- In total 0.00701024 seconds elapsed during the decomposition ---
***************************************************************
Starting to disentangle 2-qubit matrix via custom gate structure
***************************************************************


***************************************************************
Final fine tuning of the parameters 

{'time': 0.00924835205078125,
 'total_gate': 10.0,
 'cnot_gate': 2.0,
 'error': 2.6633113981615963}

## 3. 3 qubits unitary evaluation

- qiskit

In [56]:
import qiskit_decomposition

result = []
for arr in array3:
    res = qiskit_decomposition.qiskit_decomp(arr, 3)
    result.append(res)
 
print(result)

averages_3q_qiskit = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_3q_qiskit

[{'time': 0.05131196975708008, 'total_gate': 57, 'cnot_gate': 20, 'error': 3.870028700659535e-14}, {'time': 0.04373311996459961, 'total_gate': 57, 'cnot_gate': 20, 'error': 2.0913713291342845e-14}, {'time': 0.04593229293823242, 'total_gate': 57, 'cnot_gate': 20, 'error': 2.1026586472451436e-14}, {'time': 0.04355955123901367, 'total_gate': 57, 'cnot_gate': 20, 'error': 3.522287551046254e-14}, {'time': 0.04628467559814453, 'total_gate': 57, 'cnot_gate': 20, 'error': 1.8142980806656495e-14}, {'time': 0.04705238342285156, 'total_gate': 57, 'cnot_gate': 20, 'error': 4.291387549684198e-14}, {'time': 0.04344940185546875, 'total_gate': 57, 'cnot_gate': 20, 'error': 1.5587752664580726e-14}, {'time': 0.04299306869506836, 'total_gate': 57, 'cnot_gate': 20, 'error': 4.0200316196980766e-14}, {'time': 0.04360771179199219, 'total_gate': 57, 'cnot_gate': 20, 'error': 5.1908641614859755e-14}, {'time': 0.037528276443481445, 'total_gate': 57, 'cnot_gate': 20, 'error': 2.1852501957454015e-14}]


{'time': 0.04454524517059326,
 'total_gate': 57.0,
 'cnot_gate': 20.0,
 'error': 3.064695310182259e-14}

- OpenQL

In [87]:
import openQLs

result = []
for arr in array3:
    res = openQLs.openQL_decomp(arr, 3)
    result.append(res)
 
print(result)

averages_3q_openQL = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_3q_openQL

Matrix to be factorized:
 [[-0.22460748-0.36741328j -0.07562202-0.3213404j   0.04758861+0.05874553j
  -0.47841423-0.16029387j  0.49761532-0.04725452j -0.19014196+0.06321662j
   0.05343065+0.1667107j  -0.3320579 -0.11990229j]
 [ 0.21229002+0.01620005j -0.1149668 -0.44833637j -0.0154931 -0.5724281j
   0.1244403 +0.04456126j -0.16101749+0.15304484j -0.35306643-0.13337329j
   0.17722549+0.13978196j -0.12666226+0.36915807j]
 [ 0.45798865+0.09933268j -0.01469484-0.19512949j -0.08416931+0.37657437j
   0.42748497+0.23289581j  0.3815261 -0.06601414j  0.05765703+0.20837807j
  -0.05800559-0.01106704j -0.37415246+0.12676118j]
 [ 0.24616257+0.23955046j  0.05285545-0.16999078j -0.19652146+0.09357524j
  -0.16286023-0.24212697j -0.11780402-0.22985489j  0.2996295 +0.00786293j
   0.68137964+0.24191239j  0.04676874-0.19043429j]
 [ 0.00610326-0.13390162j  0.23599282-0.07964419j  0.09647735-0.0056174j
   0.25973748+0.08369192j -0.02690356-0.46805616j -0.58372741+0.14643965j
   0.16393389-0.18436651j  0.224

{'time': 0.03876066207885742,
 'total_gate': 120.0,
 'cnot_gate': 36.0,
 'error': 3.862635667603361}

- SQUANDER

In [100]:
import squander

result = []
for arr in array3:
    res = squander.squander_decomp(arr, 3)
    result.append(res)
 
print(result)

averages_3q_squander = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_3q_squander


Original circuit:
     ┌────────────────┐
q_0: ┤0               ├
     │                │
q_1: ┤1 original gate ├
     │                │
q_2: ┤2               ├
     └────────────────┘



Discretized circuit:
***************************************************************
Starting to disentangle 3-qubit matrix via custom gate structure
***************************************************************


     ┌────────────┐┌─────────────┐┌───┐┌────────────┐ ┌──────────┐┌───┐»
q_0: ┤ Rz(5.8828) ├┤ Ry(-1.7161) ├┤ X ├┤ Rz(1.8101) ├─┤ Ry(6.75) ├┤ X ├»
     ├───────────┬┘└┬────────────┤└─┬─┘├────────────┤┌┴──────────┤└─┬─┘»
q_1: ┤ Rz(2.364) ├──┤ Ry(10.871) ├──■──┤ Rz(1.6782) ├┤ Ry(4.093) ├──■──»
     └───────────┘  └────────────┘     └────────────┘└───────────┘     »
q_2: ──────────────────────────────────────────────────────────────────»
                                                                       »
«     ┌────────────┐┌───────────────┐┌───┐┌────────────┐┌────────────┐┌───┐»
«q_0: 

{'time': 0.006070184707641602,
 'total_gate': 70.0,
 'cnot_gate': 14.0,
 'error': 4.251997941442839}

## 4. 4 qubits unitary evaluation

- qiskit

In [78]:
import qiskit_decomposition

result = []
for arr in array4:
    res = qiskit_decomposition.qiskit_decomp(arr, 4)
    result.append(res)
 
print(result)

averages_4q_qiskit = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_4q_qiskit

[{'time': 0.21526360511779785, 'total_gate': 265, 'cnot_gate': 100, 'error': 3.1156879961957524e-13}, {'time': 0.20988845825195312, 'total_gate': 265, 'cnot_gate': 100, 'error': 8.561132708037005e-14}, {'time': 0.24987244606018066, 'total_gate': 265, 'cnot_gate': 100, 'error': 4.3994520289497246e-14}, {'time': 0.2031550407409668, 'total_gate': 265, 'cnot_gate': 100, 'error': 1.9749014620632084e-13}, {'time': 0.1815190315246582, 'total_gate': 265, 'cnot_gate': 100, 'error': 1.2292768422768727e-13}, {'time': 0.2756161689758301, 'total_gate': 265, 'cnot_gate': 100, 'error': 8.977961162809317e-14}, {'time': 0.21581244468688965, 'total_gate': 265, 'cnot_gate': 100, 'error': 5.583979058415635e-13}, {'time': 0.19457292556762695, 'total_gate': 265, 'cnot_gate': 100, 'error': 9.558860074473075e-14}, {'time': 0.20139288902282715, 'total_gate': 265, 'cnot_gate': 100, 'error': 2.69449610046667e-13}, {'time': 0.2178192138671875, 'total_gate': 265, 'cnot_gate': 100, 'error': 1.2425691829333256e-12}]

{'time': 0.2164912223815918,
 'total_gate': 265.0,
 'cnot_gate': 100.0,
 'error': 3.017377388617831e-13}

- OpenQL

In [79]:
import openQLs

result = []
for arr in array4:
    res = openQLs.openQL_decomp(arr, 4)
    result.append(res)
 
print(result)

averages_4q_openQL = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_4q_openQL

Matrix to be factorized:
 [[-1.24884675e-01-3.18528848e-01j  3.39085027e-02+1.35405104e-01j
  -1.06268292e-01-1.15678244e-01j  2.73417271e-01+1.40848403e-01j
   1.62531976e-01+1.05504617e-01j -2.79107209e-02-7.03313783e-02j
   1.68198371e-01-5.02496987e-01j -8.23260052e-02+7.90364102e-02j
  -1.60134972e-01-1.65324434e-01j -1.97793348e-02+2.58390929e-01j
   4.77821185e-02+6.01469083e-03j -7.33643749e-02+2.26060310e-02j
  -6.08868746e-02+1.65108339e-01j  2.14834594e-01+9.31306252e-02j
  -1.69036426e-01+3.77203917e-01j -3.79017357e-02+1.43751787e-01j]
 [ 2.54647637e-01-2.92444510e-01j -6.59839572e-02+2.11802788e-02j
   2.25128035e-01+2.44247027e-01j -7.17088932e-02-2.97918999e-02j
  -2.41897493e-01+7.06171678e-02j  2.03417752e-01+9.56186219e-02j
   7.69085162e-02-1.82084450e-01j  3.11482231e-01+1.29878404e-01j
  -2.13204520e-01-4.01165252e-04j  2.80265153e-02-1.43046829e-01j
  -1.51822985e-01+3.23335447e-01j -1.47353851e-01+1.51722874e-01j
   2.66196653e-01-8.30669438e-02j  8.96358484e-02

{'time': 0.10537896156311036,
 'total_gate': 528.0,
 'cnot_gate': 168.0,
 'error': 5.648461294188083}

- SQUANDER

In [101]:
import squander

result = []
for arr in array4:
    res = squander.squander_decomp(arr, 4)
    result.append(res)
 
print(result)

averages_4q_squander = {key: sum(item[key] for item in result) / len(result) for key in result[0]}

averages_4q_squander


Original circuit:
     ┌────────────────┐
q_0: ┤0               ├
     │                │
q_1: ┤1               ├
     │  original gate │
q_2: ┤2               ├
     │                │
q_3: ┤3               ├
     └────────────────┘


***************************************************************
Starting to disentangle 4-qubit matrix via custom gate structure
***************************************************************



Discretized circuit:
***************************************************************
Final fine tuning of the parameters in the 4-qubit decomposition
***************************************************************
The minimum with 14 layers after 0 iterations is 0.877727 calculated in 0.002109 seconds
Reached maximal number of iterations

In the decomposition with error = 1.98273 were used 14 gates with:
28 RY opeartions,
28 RZ opeartions,
14 CNOT opeartions,

--- In total 0.0050982 seconds elapsed during the decomposition ---
     ┌─────────────┐┌─────────────

{'time': 0.009490346908569336,
 'total_gate': 70.0,
 'cnot_gate': 14.0,
 'error': 5.463587870513076}